In [1]:
import os, json, math
from typing import Dict, List, Tuple
from concurrent.futures import ProcessPoolExecutor, as_completed

import xml.etree.ElementTree as ET
from xml.dom import minidom
import openslide
from openslide.deepzoom import DeepZoomGenerator
import os, json, math
from typing import List, Dict, Tuple, Optional

from draw_utils import *

# # DeepZoom 설정 (모델/추론 파이프라인 기본이 224일 가능성 높음)
# TILE_SIZE = 224
# OVERLAP   = 0

In [2]:
output_dir = "/home/sjhong/segmentation/histoplus/output/1_028_08"
wsi_path = "/home/sjhong/1-028_08.svs"
wsi_name = wsi_path.split('/')[-1][:-4]
iiss = (588, 602, 714)
ix = 0
json_path = f"{output_dir}/seg_{wsi_name}_{iiss[ix]}.json"

In [3]:
# JSON 로딩(여러 파일이면 순차 로딩 + 레코드 합치기)
with (open(json_path, "r", encoding="utf-8") if not json_path.endswith(".gz") else __import__("gzip").open(json_path, "rt", encoding="utf-8")) as f:
    data = json.load(f, parse_float=float)

In [4]:
#<!--ROI: x0=72931, y0=28727, width=8465, height=2604-->

In [5]:
# ── 사용 예 ───────────────────────────────────────────────────
slide = openslide.OpenSlide(wsi_path)
dz = DeepZoomGenerator(slide, tile_size=224, overlap=0, limit_bounds=False)
lvl = 16
x0, y0 = 72931, 28727 # 좌측 상단 좌표
w, h   = 8465, 2604

# (A) 낱개 타일 리스트
tiles = tiles_covering_base_rect(slide, dz, lvl, x0, y0, w, h,
                                 tile_size=224, overlap=0, return_spans=False)
if len(tiles) == 1:
    print("I founded this tile:",tiles[0])
    tile = tiles[0]
else:
    print(f"tile count: {len(tiles)}")
    print("first 10:", tiles[:10])  # (level, tx, ty)

'''
    # (B) 행 단위 span (더 빠르게 순회/다운로드)
    spans = tiles_covering_base_rect(slide, dz, lvl, x0, y0, w, h,
                                    tile_size=224, overlap=0, return_spans=True)
    print(f"row spans: {len(spans)} rows")
    print("first 5 spans:", spans[:5])  # (ty, tx0, tx1)
'''
print()

tile count: 120
first 10: [(162, 64), (163, 64), (164, 64), (165, 64), (166, 64), (167, 64), (168, 64), (169, 64), (170, 64), (171, 64)]



TIFFReadDirectory: Warning, Unknown field with tag 317 (0x13d) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 317 (0x13d) encountered.


In [6]:
found_tiles = [
    t for t in data.get("cell_masks", [])
    if (int(t.get("x", -1)), int(t.get("y", -1))) in tiles
]
print(found_tiles[0])
print(f"총 {len(found_tiles)}개 tile dict 매칭")

{'level': 16.0, 'x': 173.0, 'y': 66.0, 'width': 224.0, 'height': 224.0, 'masks': [{'cell_id': 0.0, 'cell_type': 'Apoptotic Body', 'confidence': 0.9999999947089947, 'coordinates': [[15.0, 3.5], [14.5, 4.0], [14.0, 4.0], [12.5, 5.5], [12.0, 5.5], [11.0, 6.5], [10.5, 6.5], [10.0, 7.0], [10.0, 8.5], [10.5, 9.0], [11.0, 9.0], [11.5, 9.5], [15.5, 9.5], [16.0, 9.0], [16.5, 9.0], [17.0, 8.5], [17.5, 8.5], [19.0, 7.0], [19.0, 6.5], [19.5, 6.0], [19.5, 4.5], [18.5, 3.5]], 'centroid': [15.0, 6.5]}, {'cell_id': 1.0, 'cell_type': 'Apoptotic Body', 'confidence': 0.9999999945945947, 'coordinates': [[28.0, 3.5], [27.5, 4.0], [25.0, 4.0], [24.5, 4.5], [23.5, 4.5], [23.0, 5.0], [22.5, 5.0], [22.0, 5.5], [22.0, 6.0], [22.5, 6.5], [22.5, 7.0], [23.5, 7.0], [24.0, 7.5], [24.5, 7.5], [25.0, 8.0], [25.5, 8.0], [26.0, 8.5], [27.0, 8.5], [27.5, 9.0], [30.0, 9.0], [31.5, 7.5], [31.5, 7.0], [32.0, 6.5], [32.0, 4.0], [31.0, 4.0], [30.5, 3.5]], 'centroid': [27.5, 6.0]}, {'cell_id': 2.0, 'cell_type': 'Apoptotic Bod

In [14]:
# cell_type = ['Cancer cell', 'Fibroblasts', 'Macrophages', 'Lymphocytes', 'Apoptotic Body', 'Neutrophils']
roi_comment = f"ROI: x0={x0}, y0={y0}, width={w}, height={h}"
cell_type = ['a']
annot_path = f"/home/sjhong/segmentation/histoplus/annotation/asap_{wsi_name}_{iiss[ix]}.xml"

tiles_to_asap_xml(
    wsi_path=wsi_path,
    tiles=found_tiles,                   # 3k 타일 dict 리스트
    xml_out_path=annot_path,
    color_map=COLOR_MAP,
    roi_comment = roi_comment,
    contained = list(COLOR_MAP.keys()),
    max_workers=45,                 # 코어에 맞게 조정
)

TIFFReadDirectory: Warning, Unknown field with tag 317 (0x13d) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 317 (0x13d) encountered.


ASAP XML saved: /home/sjhong/segmentation/histoplus/annotation/asap_1-028_08_588_1_1_1_1_1.xml
